In [2]:
import glob
import json
import more_itertools

In [195]:
! pip install more_itertools

In [144]:
len(glob.glob("*.json"))

1024

In [3]:
def read_all_result_file():
    all_result_dict={}
    for res_file in glob.glob("*.json"):
        with open(res_file,"r") as rhandle:
            res_dict = json.load(rhandle)
            all_result_dict[tuple(res_dict["alive_feature_dims"])] = res_dict
    return all_result_dict


# Overall Drop

In [4]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["overall_ood_drop"]) for key in all_result_dict.keys()]
all_result_list.sort(key=lambda x:x[-1])
all_result_list

[((4, 5, 6, 8), -4.562211990356445),
 ((1, 3, 5, 6, 7, 8, 9), -4.476959228515625),
 ((1, 4, 5, 6, 7, 8, 9), -4.373271942138672),
 ((1, 3, 4, 5, 6, 8, 9), -4.368663787841797),
 ((3, 5, 6, 8), -4.3594465255737305),
 ((5, 6, 7, 8, 9), -4.354838848114014),
 ((3, 5, 6, 8, 9), -4.3548383712768555),
 ((1, 3, 4, 5, 6, 8), -4.311059474945068),
 ((5, 6, 8), -4.29262638092041),
 ((1, 3, 4, 5, 6, 7, 8, 9), -4.288018226623535),
 ((1, 4, 5, 6, 8, 9), -4.278801918029785),
 ((3, 4, 5, 6, 9), -4.218894004821777),
 ((5, 6, 8, 9), -4.168202877044678),
 ((4, 5, 6, 9), -4.105990409851074),
 ((1, 5, 6, 8, 9), -4.103686809539795),
 ((3, 5, 6, 7, 8, 9), -4.09907865524292),
 ((4, 5, 8), -4.052995204925537),
 ((4, 5, 6, 7, 8, 9), -4.0506911277771),
 ((1, 3, 5, 6, 8), -4.050690650939941),
 ((3, 4, 5, 6, 8, 9), -4.046082496643066),
 ((1, 3, 4, 5, 6), -4.039170265197754),
 ((1, 3, 5, 6, 8, 9), -4.034562110900879),
 ((1, 3, 5, 6, 9), -4.032257080078125),
 ((4, 5, 6, 8, 9), -4.025345802307129),
 ((3, 5, 6, 7, 9), -4

In [205]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_ood_drop"]) for key in all_result_dict.keys() if len(key)==1]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[0:1000]

[((5,), -0.09224534779787064),
 ((8,), -0.06212335824966431),
 ((2,), -0.054641205817461014),
 ((0,), -0.05134955793619156),
 ((7,), -0.04814021661877632),
 ((1,), -0.04429084435105324),
 ((6,), -0.04352278262376785),
 ((9,), -0.035463832318782806),
 ((3,), -0.019835708662867546),
 ((4,), -0.018190648406744003)]

In [206]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["overall_ood_drop"]) for key in all_result_dict.keys() if len(key)==1]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[0:1000]

[((5,), -2.6751151084899902),
 ((8,), -1.615207314491272),
 ((6,), -1.5668201446533203),
 ((2,), -1.5299537181854248),
 ((0,), -1.437787652015686),
 ((7,), -1.3479260206222534),
 ((1,), -1.1958527565002441),
 ((9,), -0.8156681060791016),
 ((3,), -0.4562213122844696),
 ((4,), -0.34562233090400696)]

# Overall Average

In [171]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_all_vacc"]) for key in all_result_dict.keys() if len(key)==1]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((4,), 0.49150344729423523),
 ((6,), 0.49996399879455566),
 ((9,), 0.5038882493972778),
 ((3,), 0.5045362710952759),
 ((8,), 0.5106566548347473),
 ((5,), 0.5498992204666138),
 ((1,), 0.5561635494232178),
 ((7,), 0.5895737409591675),
 ((0,), 0.6036866307258606),
 ((2,), 0.639976978302002)]

In [201]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_all_vacc"]) for key in all_result_dict.keys()]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((3, 4, 8), 0.5181811451911926),
 ((3, 4, 6, 9), 0.5191532373428345),
 ((8, 9), 0.5201972723007202),
 ((3, 6, 9), 0.5202332735061646),
 ((4, 5, 8, 9), 0.5214213728904724),
 ((3, 4, 6, 8), 0.5233654975891113),
 ((3, 6, 8, 9), 0.525813639163971),
 ((4, 5, 8), 0.5264257192611694),
 ((4, 7, 8), 0.5312139987945557),
 ((5, 6, 8), 0.5325100421905518),
 ((5, 6, 9), 0.5328341126441956),
 ((3, 4, 8, 9), 0.5346702337265015),
 ((7, 9), 0.5378744602203369),
 ((4, 5, 6, 8), 0.5381264686584473),
 ((5, 8), 0.5385944843292236),
 ((3, 5, 6, 9), 0.5395665168762207),
 ((4, 5, 6, 7, 8), 0.5405386090278625),
 ((1, 4, 7, 8), 0.5407186150550842),
 ((4, 5, 6, 9), 0.5410426259040833),
 ((3, 8, 9), 0.5423387289047241),
 ((5, 6, 7, 8, 9), 0.5423747301101685),
 ((3, 4, 5, 6), 0.5427347421646118),
 ((3, 5, 6), 0.5429508090019226),
 ((3, 4, 7, 9), 0.5439947843551636),
 ((3, 5, 8, 9), 0.5440307855606079),
 ((3, 4, 5, 6, 9), 0.5442467927932739),
 ((4, 5, 6, 8, 9), 0.544282853603363),
 ((3, 5, 8), 0.5448588728904724),

# OOD Average

In [186]:
all_result_dict = read_all_result_file()
num_cat = len(all_result_dict[()]["data_args"]["cat_list"])
all_result_list = [
                    (
                        key,
                        (  (all_result_dict[key]["ood_meta_dict"]["mean_all_vacc"]*(num_cat**2)) - (all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]*(num_cat))    )/(num_cat*(num_cat-1))
                    ) 
                               for key in all_result_dict.keys() if len(key)==1
                  ]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((4,), 0.4922234926904951),
 ((6,), 0.49790158442088533),
 ((9,), 0.503785388810294),
 ((3,), 0.5051431655883789),
 ((8,), 0.5097926003592355),
 ((5,), 0.546329847403935),
 ((1,), 0.5561635494232178),
 ((7,), 0.5895737409591675),
 ((0,), 0.6036866307258606),
 ((2,), 0.639976978302002)]

In [188]:
all_result_dict = read_all_result_file()
num_cat = len(all_result_dict[()]["data_args"]["cat_list"])
all_result_list = [
                    (
                        key,
                        (  (all_result_dict[key]["ood_meta_dict"]["mean_all_vacc"]*(num_cat**2)) - (all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]*(num_cat))    )/(num_cat*(num_cat-1))
                    ) 
                               for key in all_result_dict.keys()
                  ]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((3, 4, 8), 0.515882134437561),
 ((3, 4, 9), 0.5160467198916844),
 ((3, 4, 6, 9), 0.5161290424210685),
 ((8, 9), 0.5185154506138393),
 ((4, 5, 8), 0.5185155017035348),
 ((3, 6, 9), 0.5189680542264666),
 ((3, 4, 6, 8), 0.519544107573373),
 ((3, 6, 8, 9), 0.5202847123146057),
 ((5, 6, 8), 0.5240289313452584),
 ((5, 6, 9), 0.5256748029163906),
 ((4, 7, 8), 0.5266622815813337),
 ((4, 5, 6, 8), 0.5295013529913766),
 ((5, 8), 0.5324638060161045),
 ((3, 4, 8, 9), 0.5330809865679059),
 ((4, 5, 6, 9), 0.5333689962114606),
 ((4, 5, 6, 7, 8), 0.5333690132413592),
 ((3, 5, 6, 9), 0.5334101234163556),
 ((5, 6, 7, 8, 9), 0.5341919234820774),
 ((3, 4, 5, 6, 9), 0.5363725679261344),
 ((4, 5, 6, 8, 9), 0.5365783657346453),
 ((7, 9), 0.5367429597037179),
 ((3, 5, 6), 0.5370310119220189),
 ((3, 5, 8), 0.5375658358846392),
 ((3, 5, 8, 9), 0.5377715229988098),
 ((3, 4, 5, 6), 0.5385944758142743),
 ((5, 7, 8), 0.5388413412230355),
 ((3, 5, 6, 8, 9), 0.5391704865864345),
 ((6, 7, 8), 0.5393350975854057),
 (

# Indo Average

In [170]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]) for key in all_result_dict.keys() if len(key)==1]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((4,), 0.48646312952041626),
 ((3,), 0.5002880096435547),
 ((9,), 0.5046082735061646),
 ((6,), 0.5144008994102478),
 ((8,), 0.5167050361633301),
 ((1,), 0.5561635494232178),
 ((5,), 0.574884831905365),
 ((7,), 0.5895737409591675),
 ((0,), 0.6036866307258606),
 ((2,), 0.639976978302002)]

In [210]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]) for key in all_result_dict.keys()]
all_result_list.sort(key=lambda x:x[-1])
all_result_list#[-1000:]

[((), 0.4861750900745392),
 ((4,), 0.48646312952041626),
 ((3,), 0.5002880096435547),
 ((9,), 0.5046082735061646),
 ((4, 9), 0.5066244602203369),
 ((3, 4), 0.5069124698638916),
 ((6,), 0.5144008994102478),
 ((4, 8), 0.5144009590148926),
 ((3, 4, 6), 0.514976978302002),
 ((4, 6), 0.5161290168762207),
 ((8,), 0.5167050361633301),
 ((3, 4, 9), 0.5181451439857483),
 ((3, 9), 0.5227534770965576),
 ((4, 6, 9), 0.5262097120285034),
 ((3, 6), 0.5264977216720581),
 ((6, 9), 0.5264977216720581),
 ((3, 6, 9), 0.5290898084640503),
 ((3, 8), 0.5313940048217773),
 ((8, 9), 0.5319700241088867),
 ((3, 4, 8), 0.5342742204666138),
 ((4, 8, 9), 0.5360022783279419),
 ((3, 4, 6, 9), 0.5403226017951965),
 ((6, 8), 0.544066846370697),
 ((4, 6, 8), 0.5443548560142517),
 ((7, 9), 0.5457949638366699),
 ((3, 4, 8, 9), 0.5457949638366699),
 ((1, 4, 7, 8), 0.550115168094635),
 ((3, 4, 6, 8), 0.5501152276992798),
 ((3, 8, 9), 0.5506912469863892),
 ((3, 6, 7), 0.5515552759170532),
 ((3, 4, 7, 9), 0.5518432855606079)

In [189]:
all_result_dict = read_all_result_file()
all_result_list = [(key,all_result_dict[key]["ood_meta_dict"]["mean_all_vacc"]-all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]) for key in all_result_dict.keys() if len(key)==1]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((5,), -0.02498561143875122),
 ((6,), -0.014436900615692139),
 ((8,), -0.006048381328582764),
 ((9,), -0.0007200241088867188),
 ((0,), 0.0),
 ((1,), 0.0),
 ((7,), 0.0),
 ((2,), 0.0),
 ((3,), 0.004248261451721191),
 ((4,), 0.00504031777381897)]

# New Causal Score

In [159]:
all_result_dict = read_all_result_file()
all_result_list = [
                    (
                        key,
                        all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]*(1-abs(all_result_dict[key]["ood_meta_dict"]["mean_ood_drop"]))
                    ) 
                               for key in all_result_dict.keys()
                  ]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((1, 4, 7, 8), 0.5097486879571758),
 ((4, 6, 8, 9), 0.511180764813143),
 ((3, 4, 8), 0.5115164960684782),
 ((6, 8, 9), 0.5129217037789076),
 ((3, 6, 8), 0.5140772885882456),
 ((3, 4, 5, 6), 0.515554892641696),
 ((3, 4, 6, 8, 9), 0.5176087109095846),
 ((3, 4, 6, 8), 0.5183956480049541),
 ((3, 8, 9), 0.5187156177097489),
 ((3, 4, 8, 9), 0.5196011178567606),
 ((3, 6, 7), 0.5198778049286918),
 ((5,), 0.5218543806425342),
 ((7, 9), 0.5219400141140618),
 ((3, 4, 7, 9), 0.5225039555365436),
 ((3, 6, 8, 9), 0.523171543855319),
 ((3, 6, 7, 8), 0.5246426149541028),
 ((3, 7, 8), 0.5247239608431582),
 ((4, 7, 8), 0.5253809908315077),
 ((1, 4, 5, 6, 8), 0.5266054064534456),
 ((5, 8), 0.5275703573925083),
 ((1, 4, 6), 0.5288954354184696),
 ((3, 4, 5, 6, 8, 9), 0.5292060842610269),
 ((5, 7, 8), 0.5293956411632714),
 ((4, 5, 8), 0.5293967661706587),
 ((4, 5, 9), 0.5296518069402794),
 ((3, 4, 6, 7, 8, 9), 0.5302785508136112),
 ((5, 6, 9), 0.5302833083938312),
 ((4, 5, 6, 7, 8, 9), 0.5304145211574522),

# New Spurious Score

In [160]:
all_result_dict = read_all_result_file()
all_result_list = [
                    (
                        key,
                        all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]*(abs(all_result_dict[key]["ood_meta_dict"]["mean_ood_drop"]))
                    ) 
                               for key in all_result_dict.keys()
                  ]
all_result_list.sort(key=lambda x:x[-1])
all_result_list[-1000:]

[((3, 4, 8, 9), 0.02619384597990937),
 ((6, 9), 0.026621433825521734),
 ((4, 7, 9), 0.026900771634026377),
 ((0, 4, 5), 0.027063104972116392),
 ((0, 2, 5, 8, 9), 0.027120668273606263),
 ((1, 3), 0.027146997658007432),
 ((0, 1, 4, 7, 9), 0.027153815578404128),
 ((0, 4, 6, 7, 9), 0.027202461291506275),
 ((0, 4, 7), 0.027379175533852163),
 ((0, 1, 3, 7, 9), 0.02754071736546937),
 ((0, 2, 7, 9), 0.02760372618008944),
 ((0, 3, 9), 0.02772604286186464),
 ((1, 9), 0.0277560441595599),
 ((0, 4, 5, 6), 0.027824543412807934),
 ((0, 1, 3, 7), 0.027933064919758888),
 ((0, 3, 4, 7, 9), 0.027936978037459426),
 ((3, 7, 9), 0.02796781118216707),
 ((1, 3, 4), 0.028014450086004672),
 ((4, 7), 0.028015950299387526),
 ((0, 1, 4, 7), 0.028090130156539495),
 ((0, 7), 0.028160215656489296),
 ((0, 5, 7), 0.028165176820154514),
 ((0, 1, 7, 9), 0.028282212065120937),
 ((7,), 0.02838220760251664),
 ((0, 5, 9), 0.028440495301634172),
 ((0, 2, 4, 5, 7, 9), 0.028444968108530366),
 ((0, 4, 9), 0.028623985721732303),

# Partition [causal,spurious] score

In [163]:
all_result_dict = read_all_result_file()
all_idx_set = set(range(10))
partition_score_list = []

for causal_idx,causal_result in all_result_dict.items():
    #Getting the other half of the partition
    spurious_idx = tuple(all_idx_set.difference(causal_idx))
    spurious_result = all_result_dict[spurious_idx]
    
    #Now we will get the overall score of this partition
#     causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
    causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
    spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
    
    partition_score_list.append([causal_idx,spurious_idx,causal_score*spurious_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-1000:]
    

[[(1, 3, 6, 8), (0, 2, 4, 5, 7, 9), 0.04086141293968973],
 [(0, 1, 2, 3, 4, 5, 7, 8, 9), (6,), 0.04086393011235334],
 [(2, 3, 5, 6, 8, 9), (0, 1, 4, 7), 0.04118448489781712],
 [(1, 3, 8, 9), (0, 2, 4, 5, 6, 7), 0.04118478227485001],
 [(0, 2, 3, 4, 5, 6, 7, 8, 9), (1,), 0.041243505459137964],
 [(0, 1, 2, 3, 4, 5, 6, 8), (7, 9), 0.041270248744172844],
 [(3, 5, 6, 8), (0, 1, 2, 4, 7, 9), 0.04144340555877746],
 [(2, 4, 5, 6, 9), (0, 1, 3, 7, 8), 0.041612723991178324],
 [(0, 2, 3, 5, 6, 7, 8, 9), (1, 4), 0.041678848897057266],
 [(2, 3, 4, 6, 9), (0, 1, 5, 7, 8), 0.04171623075113588],
 [(1, 2, 3, 7, 8, 9), (0, 4, 5, 6), 0.041720486322046],
 [(3, 5, 7, 8, 9), (0, 1, 2, 4, 6), 0.04174504528045539],
 [(1, 2, 3, 4, 5, 6, 8, 9), (0, 7), 0.041766249205142575],
 [(1, 2, 3, 5, 6, 8), (0, 4, 7, 9), 0.041860855000446395],
 [(3, 4, 5, 8, 9), (0, 1, 2, 6, 7), 0.04187367796176261],
 [(0, 1, 2, 3, 5, 7, 8, 9), (4, 6), 0.042102896479342705],
 [(1, 2, 3, 4, 6, 8, 9), (0, 5, 7), 0.04226793478770996],
 [(1, 2

In [168]:
all_result_dict = read_all_result_file()
all_idx_set = set(range(10))
partition_score_list = []

for causal_idx,causal_result in all_result_dict.items():
    #Getting the other half of the partition
    spurious_idx = tuple(all_idx_set.difference(causal_idx))
    spurious_result = all_result_dict[spurious_idx]
    
    #Now we will get the overall score of this partition
    causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
    spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
#     causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
    
    partition_score_list.append([causal_idx,spurious_idx,causal_score*spurious_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-1000:]
    

[[(1, 3, 4, 5, 6, 8), (0, 2, 7, 9), 0.015472585709392021],
 [(4, 6, 8), (0, 1, 2, 3, 5, 7, 9), 0.015577608659885212],
 [(1, 3, 6, 8), (0, 2, 4, 5, 7, 9), 0.015585886612037494],
 [(1, 3, 8, 9), (0, 2, 4, 5, 6, 7), 0.015726582903559945],
 [(5, 6, 8, 9), (0, 1, 2, 3, 4, 7), 0.015732002080211947],
 [(0, 1, 2, 4, 5, 6, 7, 8), (3, 9), 0.015779514570991835],
 [(1, 4, 7, 8), (0, 2, 3, 5, 6, 9), 0.015909720159804613],
 [(1, 2, 3, 5, 8), (0, 4, 6, 7, 9), 0.015919610218106638],
 [(3, 5, 7, 8, 9), (0, 1, 2, 4, 6), 0.015950722868927927],
 [(3, 8), (0, 1, 2, 4, 5, 6, 7, 9), 0.015950959720670302],
 [(2, 4, 5, 6, 8), (0, 1, 3, 7, 9), 0.01597310551705444],
 [(1, 2, 3, 6, 7, 8, 9), (0, 4, 5), 0.016303895462926567],
 [(1, 2, 3, 7, 8, 9), (0, 4, 5, 6), 0.016306270526278616],
 [(3, 4), (0, 1, 2, 5, 6, 7, 8, 9), 0.016376578110661552],
 [(1, 2, 4, 5, 6, 7, 8), (0, 3, 9), 0.016379823552440564],
 [(3, 5, 6, 8, 9), (0, 1, 2, 4, 7), 0.016408460501094835],
 [(1, 2, 3, 4, 6, 8, 9), (0, 5, 7), 0.01644711273362971],

In [193]:
all_result_dict = read_all_result_file()
all_idx_set = set(range(10))
partition_score_list = []

for causal_idx,causal_result in all_result_dict.items():
    #Getting the other half of the partition
    spurious_idx = tuple(all_idx_set.difference(causal_idx))
    spurious_result = all_result_dict[spurious_idx]
    
    #Now we will get the overall score of this partition
#     causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
#     causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))

    causal_score    = causal_result["ood_meta_dict"]["mean_all_vacc"]
    spurious_score  = abs(spurious_result["ood_meta_dict"]["mean_ood_drop"])
    
    
    partition_score_list.append([causal_idx,spurious_idx,causal_score*spurious_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-1000:]
    

[[(1, 3, 6, 8), (0, 2, 4, 5, 7, 9), 0.02550293483848365],
 [(3, 5, 6, 8), (0, 1, 2, 4, 7, 9), 0.025565342124554746],
 [(4, 8, 9), (0, 1, 2, 3, 5, 6, 7), 0.02561358121616486],
 [(1, 4, 7, 8), (0, 2, 3, 5, 6, 9), 0.02562071863578952],
 [(1, 3, 8, 9), (0, 2, 4, 5, 6, 7), 0.025777204456503888],
 [(1, 2, 5, 6, 8, 9), (0, 3, 4, 7), 0.025806253803623003],
 [(3, 4, 6, 8), (0, 1, 2, 5, 7, 9), 0.025875414522907647],
 [(3, 5, 8, 9), (0, 1, 2, 4, 6, 7), 0.02602017600706441],
 [(3, 5, 8), (0, 1, 2, 4, 6, 7, 9), 0.026031812123300524],
 [(1, 2, 3, 5, 8), (0, 4, 6, 7, 9), 0.026092409069052636],
 [(3, 4, 5, 6, 7, 8, 9), (0, 1, 2), 0.026107182736246237],
 [(6, 7, 8, 9), (0, 1, 2, 3, 4, 5), 0.026125127674252457],
 [(3, 4, 5, 7, 8), (0, 1, 2, 6, 9), 0.026195996785862707],
 [(3, 4, 5, 6, 8, 9), (0, 1, 2, 7), 0.02625692866676843],
 [(2, 3, 4, 5, 6, 8), (0, 1, 7, 9), 0.026405034608516953],
 [(2, 4, 5, 6, 8, 9), (0, 1, 3, 7), 0.02647545420103392],
 [(3, 6, 8), (0, 1, 2, 4, 5, 7, 9), 0.02648945496639854],
 [(1

In [175]:
all_result_dict = read_all_result_file()
all_result_list = [
                    (
                        key,
                        all_result_dict[key]["ood_meta_dict"]["mean_indo_vacc"]*(1-abs(all_result_dict[key]["ood_meta_dict"]["mean_ood_drop"]))
                    ) 
                        for key in all_result_dict.keys() if len(key)==1
                ]
all_result_list.sort(key=lambda x:x[-1])
all_result_list

[((4,), 0.477614049768466),
 ((8,), 0.4846055840923498),
 ((9,), 0.48671293030787144),
 ((3,), 0.49036444243673927),
 ((6,), 0.4920127408837449),
 ((5,), 0.5218543806425342),
 ((1,), 0.5315305962219847),
 ((7,), 0.5611915333566508),
 ((0,), 0.5726875891060987),
 ((2,), 0.6050078645121655)]

# Three way partition [causal,spurious,unwanted]

In [199]:
from more_itertools import set_partitions
all_result_dict = read_all_result_file()
all_idx_set = list(range(10))
all_three_partitions = set_partitions(all_idx_set,k=3)
partition_score_list = []

for causal_idx,spurious_idx,useless_idx in all_three_partitions:
    #Getting the other half of the partition
    causal_result   = all_result_dict[tuple(causal_idx)]
    spurious_result = all_result_dict[tuple(spurious_idx)]
    useless_result  = all_result_dict[tuple(useless_idx)]
    
    #Now we will get the overall score of this partition
#     causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
#     causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))

    causal_score    = causal_result["ood_meta_dict"]["mean_all_vacc"]          #large ood accuracy
    spurious_score  = abs(spurious_result["ood_meta_dict"]["mean_ood_drop"])   #large drops
    useless_score   = 1-useless_result["ood_meta_dict"]["mean_indo_vacc"]       #even indo acc is bad
    
    
    partition_score_list.append([tuple(causal_idx),tuple(spurious_idx),tuple(useless_idx),causal_score*spurious_score*useless_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-10:]
    

[[(0, 1, 2, 3, 4, 7), (5, 6, 8), (9,), 0.03208221559453751],
 [(0, 1, 2, 6), (5, 7, 8), (3, 4, 9), 0.03258019427385352],
 [(0, 1, 2, 4), (3, 5, 6, 8), (7, 9), 0.032713667408933424],
 [(0, 1, 2, 3), (4, 5, 6, 8), (7, 9), 0.03273483735559759],
 [(0, 1, 2, 3, 4, 6), (5, 7, 8), (9,), 0.03284655736660443],
 [(0, 1, 2, 3, 6), (5, 7, 8), (4, 9), 0.03306309482852771],
 [(0, 1, 2, 7), (4, 5, 6, 8), (3, 9), 0.03406883326774767],
 [(0, 1, 2, 3, 7), (4, 5, 6, 8), (9,), 0.03472483006866974],
 [(0, 1, 2, 7), (3, 5, 6, 8), (4, 9), 0.036243713472452636],
 [(0, 1, 2, 4, 7), (3, 5, 6, 8), (9,), 0.03644764735959215]]

In [208]:
from more_itertools import set_partitions
all_result_dict = read_all_result_file()
all_idx_set = list(range(10))
all_three_partitions = set_partitions(all_idx_set,k=3)
partition_score_list = []

for causal_idx,spurious_idx,useless_idx in all_three_partitions:
    #Getting the other half of the partition
    causal_result   = all_result_dict[tuple(causal_idx)]
    spurious_result = all_result_dict[tuple(spurious_idx)]
    useless_result  = all_result_dict[tuple(useless_idx)]
    
    #Now we will get the overall score of this partition
#     causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
#     causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))

    causal_score    = 1-abs(causal_result["ood_meta_dict"]["mean_ood_drop"])         #low drop
    spurious_score  = abs(spurious_result["ood_meta_dict"]["mean_ood_drop"])         #large drops
    useless_score   = 1-useless_result["ood_meta_dict"]["mean_indo_vacc"]            #even indo acc is bad
    
    
    partition_score_list.append([tuple(causal_idx),tuple(spurious_idx),tuple(useless_idx),causal_score*spurious_score*useless_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-100:]
    

[[(1, 2, 7), (3, 5, 6, 8), (0, 4, 9), 0.0412958758291121],
 [(0, 1, 2, 3, 7), (5, 8), (4, 6, 9), 0.04130810972052006],
 [(0, 2, 6, 7), (3, 4, 5, 8), (1, 9), 0.041333911518891195],
 [(0, 1, 2, 3), (4, 5, 6, 7, 8), (9,), 0.04133645450940697],
 [(0, 2, 4, 6), (5, 7, 8), (1, 3, 9), 0.041345745683551795],
 [(0, 1, 2, 3), (5, 6, 7), (4, 8, 9), 0.0413636577700382],
 [(1, 2, 4, 7), (3, 5, 6, 8), (0, 9), 0.04136827160489023],
 [(0, 2, 3, 4, 6, 7), (1, 5, 8), (9,), 0.04139736162878466],
 [(0, 2, 3, 4, 7), (5, 6, 8), (1, 9), 0.04142257778791801],
 [(0, 2, 7), (1, 4, 5, 8), (3, 6, 9), 0.04156048405633666],
 [(0, 3, 7), (1, 2, 5, 6, 8), (4, 9), 0.041606084157741606],
 [(0, 1, 2), (3, 5, 7, 8), (4, 6, 9), 0.041635513726175556],
 [(0, 1, 3, 4, 7), (2, 5, 6, 8), (9,), 0.04166602322903813],
 [(0, 7), (1, 2, 3, 5, 6, 8), (4, 9), 0.04169696220896041],
 [(0, 1, 2, 4, 7), (5, 8), (3, 6, 9), 0.041704549669645825],
 [(0, 2, 3, 7), (1, 4, 5, 8), (6, 9), 0.04170496303658092],
 [(0, 2, 3, 6), (5, 7, 8), (1, 4, 

In [209]:
from more_itertools import set_partitions
all_result_dict = read_all_result_file()
all_idx_set = list(range(10))
all_three_partitions = set_partitions(all_idx_set,k=3)
partition_score_list = []

for causal_idx,spurious_idx,useless_idx in all_three_partitions:
    #Getting the other half of the partition
    causal_result   = all_result_dict[tuple(causal_idx)]
    spurious_result = all_result_dict[tuple(spurious_idx)]
    useless_result  = all_result_dict[tuple(useless_idx)]
    
    #Now we will get the overall score of this partition
#     causal_score = causal_result["ood_meta_dict"]["mean_indo_vacc"] * (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = spurious_result["ood_meta_dict"]["mean_indo_vacc"] * (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))
    
#     causal_score =  (1- abs(causal_result["ood_meta_dict"]["mean_ood_drop"]))
#     spurious_score = (abs(spurious_result["ood_meta_dict"]["mean_ood_drop"]))

    causal_score    = causal_result["ood_meta_dict"]["mean_all_vacc"]* ( 1-abs(causal_result["ood_meta_dict"]["mean_ood_drop"]) )        #low drop
    spurious_score  = spurious_result["ood_meta_dict"]["mean_all_vacc"]* abs(spurious_result["ood_meta_dict"]["mean_ood_drop"])         #large drops
    useless_score   = 1-useless_result["ood_meta_dict"]["mean_indo_vacc"]            #even indo acc is bad
    
    
    partition_score_list.append([tuple(causal_idx),tuple(spurious_idx),tuple(useless_idx),causal_score*spurious_score*useless_score])

partition_score_list.sort(key=lambda x: x[-1])
partition_score_list[-100:]
    

[[(0, 2, 7), (1, 4, 5, 8), (3, 6, 9), 0.014843997711124978],
 [(0, 4, 7), (1, 2, 3, 5, 6, 8), (9,), 0.01487613519858359],
 [(0, 1, 2, 6, 7), (5, 8), (3, 4, 9), 0.01487973366057789],
 [(0, 2, 4), (1, 3, 5, 6, 7), (8, 9), 0.014889959462548858],
 [(0, 1, 6, 7), (2, 3, 4, 5, 8), (9,), 0.014894799772362431],
 [(0, 2), (1, 3, 4, 5, 6, 7), (8, 9), 0.01490506885013663],
 [(0, 2), (1, 4, 5, 6, 7), (3, 8, 9), 0.014906641046469896],
 [(0, 2, 7), (4, 5, 6, 8), (1, 3, 9), 0.014971105794155367],
 [(0, 1, 3), (2, 4, 5, 6, 8), (7, 9), 0.014976373533354058],
 [(0, 1, 2, 3, 4, 6, 7), (5, 8), (9,), 0.01497849602430448],
 [(0, 1, 2, 5), (3, 4, 6, 7, 8), (9,), 0.014978501816452227],
 [(0, 1, 2), (5, 6, 8), (3, 4, 7, 9), 0.01501409534543174],
 [(0, 1, 2), (5, 6, 7), (3, 4, 8, 9), 0.015020081264918248],
 [(0, 2, 6, 7), (1, 4, 5, 8), (3, 9), 0.015021994253256593],
 [(0, 1, 2, 4, 6, 7), (5, 8), (3, 9), 0.015028176919592285],
 [(0, 1, 4, 7), (2, 5, 6, 8), (3, 9), 0.015054533181033782],
 [(0, 2, 3, 7), (1, 4, 5,